IPOPT Tutorial Example

Victor M. Zavala 2016 

University of Wisconsin-Madison  

In [20]:
using JuMP 
using Ipopt 

In [21]:
# Model 
m = Model(solver=IpoptSolver(tol = 1e-8, max_iter = 100,linear_solver ="mumps",mu_strategy="monotone"))

NS=3
S=collect(1:3)
Cai=zeros(NS)
pr=zeros(NS)

Cai[1]=15
Cai[2]=10
Cai[3]=20
pr[1]=0.7
pr[2]=0.2
pr[3]=0.1

Ti=20
Tci=15
rol= 800.8
cel=0.968
ror=1041.1
cer=1.291
U=6129
Ea=6867.36
R=8.314
Beta=5.967
H=69710.5
V=38.3
A=24

@variable(m,  0.5 <= q <= 15, start= 2.83)    
@variable(m, 20 <= T[S] <= 200, start = 73.3)
@variable(m, 10 <= Tc[S] <= 69, start =63.31)
@variable(m, 0.9 <= Ca[S] <= 16, start = 1.866)
@variable(m, 5 <= Cb[S] <= 30, start = 13.1339)
@variable(m, 20 <= F[S] <= 600, start = 450)
@variable(m, 0.7 <= x[S] <= 0.9, start = 0.87) 
@variable(m, k[S], start = 0.54) 
@variable(m, cost[S]) 

@NLconstraint(m, balca[s in S], q*(Cai[s] - Ca[s]) - V*k[s]*Ca[s] == 0) 
@NLconstraint(m, balcb[s in S], V*k[s]*Ca[s] - q*Cb[s] == 0) 

@NLconstraint(m, balcT[s in S], 
    q*rol*cel*(Ti - T[s]) + V*k[s]*Ca[s]*H - (U*A)*(T[s]- Tc[s]) == 0) 

@NLconstraint(m, balcTc[s in S], F[s]*ror*cer*(Tci - Tc[s]) + U*A*(T[s] - Tc[s]) == 0) 

@NLconstraint(m, arrhenius[s in S], k[s] - Beta*exp(-Ea/(R*(T[s]+273.15))) == 0) 

@NLconstraint(m, conv[s in S], Ca[s] - Cai[s]*(1-x[s]) == 0) 

@constraint(m, approach[s in S], T[s] - Tc[s] >= 5) 
@constraint(m, restime1, V <= 25*q) 
@constraint(m, restime2, 15*q <= V) 

@NLconstraint(m, costeq[s in S], cost[s]==100*q*Cb[s] - 80*q*Cai[s]) 

@NLobjective(m, Max, sum(pr[s]*cost[s] for s in S))

# print model
#println("\n")
#print(m)
#println("\n")

In [22]:
# solve model and get solution
solve(m)

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       66
Number of nonzeros in inequality constraint Jacobian.:        8
Number of nonzeros in Lagrangian Hessian.............:       72

Total number of variables............................:       25
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       19
                     variables with only upper bounds:        0
Total number of equality constraints.................:       21
Total number of inequality constraints...............:        5
        inequality constraints with only lower bounds:        3
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

:Optimal

In [23]:
println("Ca = ", getvalue(Ca), "\n T = ", getvalue(T))
println("\n q = ", getvalue(q), "\n F = ", getvalue(F))
println("\n cost = ", getvalue(cost), "\n Tc= ", getvalue(Tc))

Ca = Ca: 1 dimensions:
[1] = 1.5173522955195395
[2] = 1.0442588763924474
[3] = 1.9999998000980004
 T = T: 1 dimensions:
[1] = 84.44754985496282
[2] = 79.04247615630797
[3] = 86.43834251412198

 q = 2.5533333588011504
 F = F: 1 dimensions:
[1] = 31.307471587966386
[2] = 20.353035438035104
[3] = 45.219488715005554

 cost = cost: 1 dimensions:
[1] = 378.5693843199906
[2] = 244.03256932852534
[3] = 510.66672280187294
 Tc= Tc: 1 dimensions:
[1] = 69.0
[2] = 69.0
[3] = 65.55133909889908
